In [1]:
import torch

!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

ImportError: /home/qvr905/.conda/envs/mistralEnv/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so: symbol cudaLaunchKernelExC, version libcudart.so.11.0 not defined in file libcudart.so.11.0 with link time reference

In [55]:
from unsloth import FastLanguageModel
import torch
from transformers import  TrainingArguments

max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla V100S-PCIE-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [56]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

In [57]:
pythonCodes_prompt = """Below is an instruction that describes a code task. Write a response that 
appropriately return the python code.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = pythonCodes_prompt.format(instruction, output)
        texts.append(text)
    return { "text" : texts}
pass

from datasets import load_dataset
dataset = load_dataset("flytech/python-codes-25k", split = "train")
dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
#dataset = dataset.map(formatting_prompts_func, batched = True,)
print(len(dataset["train"]))
print(len(dataset["test"]))

for i in range(5):
    print(dataset["train"][i])

44663
4963
{'input': '', 'output': "```python\nimport pandas as pd\ndf = pd.read_csv('data.csv')\nmean_age = df['age'].mean()\n# Code executed. Mean age calculated.\n```", 'text': "Execute code: import pandas as pd\ndf = pd.read_csv('data.csv')\ndf['age'].mean() Calculating the mean age from a Pandas DataFrame... ```python\nimport pandas as pd\ndf = pd.read_csv('data.csv')\nmean_age = df['age'].mean()\n# Code executed. Mean age calculated.\n```", 'instruction': "Execute code: import pandas as pd\ndf = pd.read_csv('data.csv')\ndf['age'].mean()"}
{'input': '', 'output': '```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom matplotlib import style\nfrom sklearn import preprocessing\nfrom sklearn.model_selection import cross_validate, train_test_split\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import accuracy_score\n\nstyle.use("seaborn-whitegrid")\n\n# Load the dataset\ndf = pd.read_csv("dataset.csv")\n\n# Preprocess the data\nle = preproce

In [58]:
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [50]:
!pip install "git+https://github.com/huggingface/transformers.git"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-yr26va8u
  Running command git clone --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-yr26va8u
  Resolved https://github.com/huggingface/transformers.git to commit 76a33a10923ccc1074917f6b6a1e719e626b7dc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [59]:
trainer.train()

/tmp/tmpyg_odvfk/main.c: In function ‘list_to_cuuint64_array’:
/tmp/tmpyg_odvfk/main.c:354:3: error: ‘for’ loop initial declarations are only allowed in C99 mode
   for (Py_ssize_t i = 0; i < len; i++) {
   ^
/tmp/tmpyg_odvfk/main.c:354:3: note: use option -std=c99 or -std=gnu99 to compile your code
/tmp/tmpyg_odvfk/main.c: In function ‘list_to_cuuint32_array’:
/tmp/tmpyg_odvfk/main.c:365:3: error: ‘for’ loop initial declarations are only allowed in C99 mode
   for (Py_ssize_t i = 0; i < len; i++) {
   ^


CalledProcessError: Command '['/usr/bin/gcc', '/tmp/tmpyg_odvfk/main.c', '-O3', '-I/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/triton/common/../third_party/cuda/include', '-I/home/qvr905/.conda/envs/mis/include/python3.10', '-I/tmp/tmpyg_odvfk', '-shared', '-fPIC', '-lcuda', '-o', '/tmp/tmpyg_odvfk/cuda_utils.cpython-310-x86_64-linux-gnu.so', '-L/lib64', '-L/lib64']' returned non-zero exit status 1.